# ML @ AMSI Bioinfosummer

This is a notebook containing some code for ML @ AMSI Bioinfosummer

In [8]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn import datasets
from sklearn import svm
import matplotlib.pyplot as plt

print("Here is some code")


Here is some code


This is full of great examples.

In [47]:
X,y=datasets.load_diabetes(return_X_y=True)
y=y>100
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=0)
X_train.shape, y_train.shape


((397, 10), (397,))

In [48]:
X_test.shape, y_test.shape

((45, 10), (45,))

In [52]:
clf = sklearn.linear_model.LogisticRegression(penalty='none').fit(X_train, y_train)
#clf.score(X_test, y_test)


clf.score(X_train, y_train)


/home/bwgoudey/.local/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


0.7808564231738035